In [1]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# unpickle all variables
with open('processed_dataset.pickle' , 'rb') as f:
    dataset = pickle.load(f)
    diag = pickle.load(f)
    localization = pickle.load(f)

In [5]:
# import image, image_id.jpg , as a np array and save it to the dataset
list_images = list(dataset.index.values)
for i in range(len(list_images)):
    image = tf.keras.preprocessing.image.load_img('images/'+list_images[i]+'.jpg', target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = tf.keras.applications.vgg16.preprocess_input(image)
    dataset.loc[list_images[i],'image'] = image


FileNotFoundError: [Errno 2] No such file or directory: 'images/ISIC_0052212.jpg'